In [2]:
# -*- coding: utf-8
# Transparency International - Brazil (https://transparenciainternacional.org.br)
# Reinaldo Chaves (@paidatocandeira)
# Raspa site do STF em busca de INQs e APs de políticos
#

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import unidecode
import matplotlib.pyplot as plt
import re
%matplotlib inline

Acessa candidatos

In [6]:
eleitos = pd.read_csv('resultados/candidatos_2010_2014.csv', encoding='utf-8', converters={'sequencial': lambda x: str(x), 
                                                                                          'cpf': lambda x: str(x)})

In [7]:
eleitos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594 entries, 0 to 593
Data columns (total 9 columns):
cargo              594 non-null object
cpf                594 non-null object
nome_completo      594 non-null object
nome_urna          594 non-null object
partido_eleicao    594 non-null object
sequencial         594 non-null object
situacao           594 non-null object
situacao_cand      54 non-null object
uf                 594 non-null object
dtypes: object(9)
memory usage: 41.8+ KB


In [9]:
def f(str):
    return (unidecode.unidecode(str))

In [10]:
eleitos["nome_completo"] = eleitos["nome_completo"].apply(f)

In [13]:
def le_lista(politico_link, nome_urna, nome, cpf):
    print(nome_urna)
    try:
        res = requests.get(politico_link)
    except (requests.exceptions.HTTPError, requests.exceptions.RequestException, requests.exceptions.ConnectionError, requests.exceptions.Timeout) as e:
        print(str(e))
        return 
    except Exception as e:
        print("Exceção")
        return
        
    soup =  BeautifulSoup(res.text, "lxml")

    parts = soup.find_all('table', {'class': 'table m-b-0'})
    
    for part in parts:
        lines = part.find_all('td')
        
        chunks = [lines[x:x+5] for x in range(0, len(lines), 5)]
    
        for vez in chunks:
            processos.append({'nome_completo': nome, 
                                'nome_urna': nome_urna, 
                                'cpf': cpf, 
                                'identificacao': vez[0].string, 
                                'identificacao_link': "http://portal.stf.jus.br/processos/" + vez[0].a.get('href'), 
                                'numero_unico': vez[1].string, 
                                'data_autuacao': vez[2].string, 
                                'meio': vez[3].string, 
                                'publicidade': vez[4].string})    
 
    return processos

In [14]:
link_stf = 'http://portal.stf.jus.br/processos/listarPartes.asp?termo='
    
processos = []
    
for num, row in eleitos.iterrows():
    nome = row['nome_completo']
    nome_urna = row['nome_urna']
    cpf = row['cpf']
    politico_link = link_stf + nome
    le_lista(politico_link, nome_urna, nome, cpf)

LUIZ HENRIQUE DA SILVEIRA
PAULO BAUER
IVO CASSOL
VALDIR RAUPP
AÉCIO NEVES
ITAMAR FRANCO
LIDICE
WALTER PINHEIRO
EUNICIO
PIMENTEL
VALADARES
EDUARDO AMORIM
CRISTOVAM BUARQUE
ROLLEMBERG
JORGE VIANA
PETECAO
LOBÃO
JOÃO ALBERTO
DELCÍDIO
MOKA
ALOYSIO NUNES
MARTA SUPLICY
DEMOSTENES TORRES
LÚCIA VÂNIA
CASSIO CUNHA LIMA
VITALZINHO
ANGELA PORTELA
ROMERO JUCÁ
BLAIRO MAGGI
PEDRO TAQUES
CIRO NOGUEIRA
WELLINGTON DIAS
GARIBALDI ALVES FILHO
JOSÉ AGRIPINO
JOÃO RIBEIRO
VICENTINHO ALVES
GLEISI
REQUIÃO
LINDBERG
MARCELO CRIVELLA
MAGNO MALTA
RICARDO FERRAÇO
ANA AMÉLIA LEMOS
PAIM
ARMANDO MONTEIRO
HUMBERTO COSTA
EDUARDO BRAGA
VANESSA GRAZZIOTIN
CAPI
RANDOLFE
BENEDITO DE LIRA
RENAN
FLEXA RIBEIRO
JADER BARBALHO
PAULO ROCHA
EDMILSON RODRIGUES
BETO SALAME
PRIANTE
CHAPADINHA
JOSUÉ BENGTSON
SIMONE MORGADO
WLAD
NILSON PINTO
JORDY
LUCIO VALE
JOAQUIM PASSARINHO
ZÉ GERALDO
BETO FARO
HÉLIO LEITE
ELCIONE
JULIA MARINHO
DELEGADO EDER MAURO
SANDRO ALEX
ALFREDO KAEFER
ZECA DIRCEU
HERMES FRANGÃO PARCIANELLO
ALIEL MACHADO
DILCEU

In [17]:
df_processos = pd.DataFrame(processos, columns = ['nome_completo', 'nome_urna', 'cpf', 'identificacao','identificacao_link', 'numero_unico','data_autuacao', 'meio', 'publicidade'])

In [18]:
df_processos.reset_index().head()

,index,nome_completo,nome_urna,cpf,identificacao,identificacao_link,numero_unico,data_autuacao,meio,publicidade
0,0,LUIZ HENRIQUE DA SILVEIRA,LUIZ HENRIQUE DA SILVEIRA,00586935991,AI 241429,http://portal.stf.jus.br/processos/detalhe.asp...,,08/04/1999,Físico,Público
1,1,LUIZ HENRIQUE DA SILVEIRA,LUIZ HENRIQUE DA SILVEIRA,00586935991,AI 246056,http://portal.stf.jus.br/processos/detalhe.asp...,,21/06/1999,Físico,Público
2,2,LUIZ HENRIQUE DA SILVEIRA,LUIZ HENRIQUE DA SILVEIRA,00586935991,AI 260099,http://portal.stf.jus.br/processos/detalhe.asp...,,07/11/1999,Físico,Público
3,3,LUIZ HENRIQUE DA SILVEIRA,LUIZ HENRIQUE DA SILVEIRA,00586935991,AI 293236,http://portal.stf.jus.br/processos/detalhe.asp...,,01/09/2000,Físico,Público
4,4,LUIZ HENRIQUE DA SILVEIRA,LUIZ HENRIQUE DA SILVEIRA,00586935991,AI 206229,http://portal.stf.jus.br/processos/detalhe.asp...,,30/10/1997,Físico,Público


In [14]:
df_processos.to_excel('resultados/processos_stf_deputados_14_jul_2018.xlsx',sheet_name='Sheet1')

Seleciona apenas Ações Penais

In [30]:
mask = df_processos['identificacao'].str.contains("AP ", na=False)
ap = df_processos[mask]

mask = df_processos['identificacao'].str.contains("Inq ", na=False)
inq = df_processos[mask]

casos = pd.concat([ap, inq], ignore_index=True)

In [45]:
casos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 9 columns):
nome_completo         1470 non-null object
nome_urna             1470 non-null object
cpf                   1470 non-null object
identificacao         1470 non-null object
identificacao_link    1470 non-null object
numero_unico          1470 non-null object
data_autuacao         1470 non-null object
meio                  1470 non-null object
publicidade           1470 non-null object
dtypes: object(9)
memory usage: 103.4+ KB


In [35]:
casos.to_excel('resultados/processos_stf.xlsx',sheet_name='Sheet1')

In [46]:
df_processos = pd.read_excel("/home/reinaldo/Downloads/processos_parlamentares_revisao_reinaldo.xlsx",sheet_name='Planilha1')

In [47]:
df_processos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1426 entries, 0 to 1425
Data columns (total 25 columns):
nome                               1426 non-null object
nome_completo                      1426 non-null object
partido                            1426 non-null object
uf                                 1426 non-null object
cargo_parlamentar                  1426 non-null object
tipo                               170 non-null object
classe                             1426 non-null object
numero                             1426 non-null object
único                              1362 non-null object
assunto                            1426 non-null object
data_inicial                       1426 non-null object
data_final                         1426 non-null object
andamento                          1426 non-null object
link                               1424 non-null object
transparencia                      1424 non-null object
conferencia                        173 non-null object

In [48]:
df_processos["nome_completo"] = df_processos["nome_completo"].apply(f)

In [59]:
final = pd.merge(casos, df_processos, left_on='nome_completo', right_on='nome_completo', how='left')

In [60]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9519 entries, 0 to 9518
Data columns (total 33 columns):
nome_completo                      9519 non-null object
nome_urna                          9519 non-null object
cpf                                9519 non-null object
identificacao                      9519 non-null object
identificacao_link                 9519 non-null object
numero_unico                       9519 non-null object
data_autuacao                      9519 non-null object
meio                               9519 non-null object
publicidade                        9519 non-null object
nome                               9362 non-null object
partido                            9362 non-null object
uf                                 9362 non-null object
cargo_parlamentar                  9362 non-null object
tipo                               2031 non-null object
classe                             9362 non-null object
numero                             9362 non-null obje

In [61]:
final.to_excel('resultados/processos_stf_teste.xlsx',sheet_name='Sheet1')

Verifica maio

In [19]:
df_processos = pd.read_excel("resultados/processos_stf_deputados_17_mai_2018.xlsx",sheet_name='Sheet1')

In [20]:
mask = df_processos['identificacao'].str.contains("AP ", na=False)
aps = df_processos[mask]

In [21]:
aps.to_excel('resultados/processos_AP_stf_deputados_17_mai_2018.xlsx',sheet_name='Sheet1')

In [23]:
df_processos = pd.read_excel("resultados/processos_stf_deputados_14_jul_2018.xlsx",sheet_name='Sheet1')

In [24]:
df_processos['nome_urna'].value_counts()

SERGIO SOUZA                   1354
JOÃO RODRIGUES                 1344
DR. JORGE SILVA                1257
TONINHO PINHEIRO               1229
WOLNEY QUEIROZ                  889
PEDRO CHAVES                    298
TADEU ALENCAR                   283
PAULINHO DA FORÇA               233
ZÉ SILVA                        220
PAULO MALUF                     137
RODRIGO GARCIA                  136
EDUARDO CUNHA                   104
GORETE PEREIRA                   95
LUIZ CARLOS RAMOS DO CHAPEU      80
JOSÉ ROCHA                       74
DR JOÃO                          64
SERGIO ZVEITER                   59
ROBERTO BRITTO                   59
VICENTINHO                       56
CAETANO                          49
RUBENS BUENO                     49
ALFREDO NASCIMENTO               49
WLAD                             47
CHICO LOPES                      46
BETO MANSUR                      39
PAULÃO                           38
PENINHA                          36
ROBERTO GÓES                

In [4]:
nome = df_processos[(df_processos['nome_urna'] == 'SERGIO SOUZA')]
nome.reset_index()

,index,nome_completo,nome_urna,identificacao,identificacao_link,numero_unico,data_autuacao,meio,publicidade
0,6152,SERGIO DE SOUZA,SERGIO SOUZA,AI 137344,http://portal.stf.jus.br/processos/detalhe.asp...,,05/03/1991,Físico,Público
1,6153,SERGIO DE SOUZA,SERGIO SOUZA,RE 198241,http://portal.stf.jus.br/processos/detalhe.asp...,,25/01/1996,Físico,Público
2,6154,SERGIO DE SOUZA,SERGIO SOUZA,RE 200432,http://portal.stf.jus.br/processos/detalhe.asp...,,15/04/1996,Físico,Público
3,6155,SERGIO DE SOUZA,SERGIO SOUZA,RE 122713,http://portal.stf.jus.br/processos/detalhe.asp...,,16/04/1990,Físico,Público
4,6156,SERGIO DE SOUZA,SERGIO SOUZA,RE 123602,http://portal.stf.jus.br/processos/detalhe.asp...,,03/05/1990,Físico,Público
5,6157,SERGIO DE SOUZA,SERGIO SOUZA,AI 622542,http://portal.stf.jus.br/processos/detalhe.asp...,,13/11/2006,Físico,Público
6,6158,SERGIO DE SOUZA,SERGIO SOUZA,AI 692811,http://portal.stf.jus.br/processos/detalhe.asp...,,21/11/2007,Físico,Público
7,6159,SERGIO DE SOUZA,SERGIO SOUZA,RE 150322,http://portal.stf.jus.br/processos/detalhe.asp...,,23/06/1992,Físico,Público
8,6160,SERGIO DE SOUZA,SERGIO SOUZA,SE 4272,http://portal.stf.jus.br/processos/detalhe.asp...,0003838-05.1989.0.01.0000,14/08/1989,Físico,Público
9,6161,SERGIO DE SOUZA,SERGIO SOUZA,AI 146897,http://portal.stf.jus.br/processos/detalhe.asp...,,31/08/1992,Físico,Público
